## 1. 라벨링 특징 파라미터

### 1.1 라벨링

In [4]:
'''
    1. 영상의 처리는 왼쪽에서 오른쪽으로 그리고 위에서 아래로 화소를 읽으면서 처리, 가장 먼저 마주친 화소에 번호 부여
    2. 현재 화소 p에 연결되고 있는 주위 화소에 같은 라벨 붙임
    3. 2단계에서 라벨을 붙인 화소에 연결되어 있는 화소에 라벨 붙임
    4. 라벨을 붙여야 할 화소가 없어질 때까지 3단계를 반복
'''

# 현재 주목 화소가 라벨이 있으면 연결되어 있는 주위 8근방 화소에 같은 라벨 붙여줌
def labelset(img, xs, ys, label):
    height, width = img.shape
    img[ys, xs] = label

    while True:
        cnt = 0
        for y in range(1, height-1):
            for x in range(1, width-1):
                if img[y, x] == label:
                    if img[y , x+1] == 255:
                        img[y , x+1] = label; cnt = cnt+1

                    if img[y-1, x+1] == 255:
                        img[y-1, x+1] = label; cnt = cnt+1

                    if img[y-1, x ] == 255:
                        img[y-1, x ] = label; cnt = cnt+1

                    if img[y-1, x-1] == 255:
                        img[y-1, x-1] = label; cnt = cnt+1

                    if img[y , x-1] == 255:
                        img[y , x-1] = label; cnt = cnt+1

                    if img[y+1, x-1] == 255:
                        img[y+1, x-1] = label; cnt = cnt+1

                    if img[y+1, x ] == 255:
                        img[y+1, x ] = label; cnt = cnt+1

                    if img[y+1, x+1] == 255 :
                        img[y+1, x+1] = label; cnt = cnt+1

        if cnt == 0:
            return (0, img)

    return (1, img)

In [5]:
# 모든 화소에 대해 라벨링 수행
def labeling(img_in, base=100):
    height, width = img_in.shape
    img_label = img_in.copy()

    label = base # 베이스 값부터 라벨링

    for y in range(1, height-1):
        for x in range(1, width-1):
            if img_label[y, x] == 255:
                if label >= 255:
                    print('error! too many labels!')
                    return -1

                _, img_label = labelset(img_label, x, y, label)
                label = label + 1

    cnt = label - base

    return img_label, cnt

In [6]:
import cv2

img = cv2.imread('images/shape.png', 0)
_, bin_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
labeled_img, cnt = labeling(bin_img)

print("객체 수", cnt)
print(labeled_img[80, 80], end=" ")
print(labeled_img[69,173], end=" ")
print(labeled_img[160,190], end=" ")
print(labeled_img[160,60])

cv2.imshow('image', img)
cv2.imshow('Labeling', labeled_img)

cv2.waitKey()
cv2.destroyAllWindows()

객체 수 4
100 101 102 103


## 2. 객체 탐지

## 3. OpenCV와 머신러닝을 이용한 필기체 숫자 인식

### 3.1 필기체 숫자 데이터 불러오기

In [7]:
import cv2
import numpy as np

img = cv2.imread('images/digits.png', cv2.IMREAD_GRAYSCALE)
print(img.shape)

(1000, 2000)


In [8]:
# 20*20 배열로 생성
cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
x = np.array(cells)
print(x.shape)

(50, 100, 20, 20)


In [9]:
X = x[:, :].reshape(-1, 400).astype(np.float32)
y = np.repeat(np.arange(10), 500)

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3)

### 3.2 머신러닝 분류모형 만들기

In [10]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(train_X, train_y)
print(model.score(test_X, test_y))

0.9426666666666667


### 3.3 머신러닝 모형 저장하고 불러오기

In [11]:
import pickle
with open('number.model', 'wb') as f:
    pickle.dump(model, f)

In [12]:
with open('number.model', 'rb') as f:
    model = pickle.load(f)

print(model.score(test_X, test_y))

0.9426666666666667


### 3.4 웹 카메라를 이용한 실시간 필기체 숫자 인식하기

In [21]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            try:
                for contour in contours:
                    (x, y), radius = cv2.minEnclosingCircle(contour)
                    if radius > 5:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)

                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (255, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        
                        dst = cv2.resize(roi, dsize=(50, 50), interpolation=cv2.INTER_AREA)
                        dst = cv2.resize(roi, dsize=(16, 16), interpolation=cv2.INTER_AREA)

                        A = np.zeros((20, 20))
                        A[2:-2, 2:-2] = dst[:, :]
                        A = A.reshape(-1, 400)

                        num = model.predict(A)
                        cv2.putText(bin_img, str(num), (xs, ys), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0))

            except Exception as e:
                print(e)

            cv2.imshow('Image', bin_img)
            if cv2.waitKey(1) & 0xFF == 27:
                break

        else:
            print('No Frame')
            break

else:
    print('Camera not opened')

cap.release()
cv2.destroyAllWindows()

No Frame
